API for common tasks in machine learning

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

The code below loads the FashionMNIST dataset using Pytorch's torchvision.datasets module.

**About FashionMNIST**
It's a dataset of 70,000 grayscale images (28×28 pixels) across 10 clothing categories:

T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

## DataLoader Setup
This code below wraps the datasets into DataLoaders, which handle batching and iteration during training/testing.

batch_size = 64
Instead of feeding all 60,000 images at once, the data is split into batches of 64 images. This is more memory-efficient and helps the model generalize better.

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Neural Network Definition

Below we create a model that will be trained to predict the clothes' categories.

In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Loss Function & Optimizer
These two lines below define how the model learns.

loss_fn = nn.CrossEntropyLoss()
The loss function measures how wrong the model's predictions are.

Takes the model's 10 output logits and the correct label
Returns a single number — the error score
Lower = better
Example: if the model predicts "Bag" with 90% confidence but the answer is "Sneaker", the loss will be high. If it predicts "Sneaker" correctly with high confidence, the loss will be low.

CrossEntropyLoss is commonly used for multi-class classification problems like this one (10 categories).

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
The optimizer is responsible for updating the model's weights to reduce the loss.

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## The train Function
This function performs one full pass through the training data, updating the model's weights on each batch.

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## The test Function
This function evaluates the model's performance on unseen data — no learning happens here, only measurement.

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## The Training Loop
This is the main training loop that ties everything together by running train and test repeatedly to improve accuracy.

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302256  [   64/60000]
loss: 2.295820  [ 6464/60000]
loss: 2.273208  [12864/60000]
loss: 2.264461  [19264/60000]
loss: 2.242330  [25664/60000]
loss: 2.217479  [32064/60000]
loss: 2.219109  [38464/60000]
loss: 2.186470  [44864/60000]
loss: 2.181127  [51264/60000]
loss: 2.145282  [57664/60000]
Test Error: 
 Accuracy: 41.3%, Avg loss: 2.145149 

Epoch 2
-------------------------------
loss: 2.153536  [   64/60000]
loss: 2.147100  [ 6464/60000]
loss: 2.085178  [12864/60000]
loss: 2.096226  [19264/60000]
loss: 2.038590  [25664/60000]
loss: 1.986362  [32064/60000]
loss: 2.003953  [38464/60000]
loss: 1.926739  [44864/60000]
loss: 1.929425  [51264/60000]
loss: 1.850182  [57664/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 1.855980 

Epoch 3
-------------------------------
loss: 1.889800  [   64/60000]
loss: 1.863339  [ 6464/60000]
loss: 1.742610  [12864/60000]
loss: 1.776304  [19264/60000]
loss: 1.666419  [25664/60000]
loss: 1.634988  [32064/600

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

## Making a Prediction
This is the inference step — using the trained model to classify a real image.

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
